In [5]:
# %% [markdown]
# # Análise Exploratória de Dados de Arboviroses
# 
# Este notebook realiza uma análise exploratória completa dos dados climáticos e de casos de arboviroses (dengue, zika e chikungunya) para os municípios de Teófilo Otoni e Diamantina.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data_loader import load_data
from src.preprocessor import DataPreprocessor
import logging
import warnings
from scipy import stats

# Configurações
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 14
pd.set_option('display.max_columns', None)

# %% [markdown]
# ## 1. Carregamento e Pré-processamento de Dados

# %%
# Carregar dados brutos
logging.info("Carregando dados...")
df_raw = load_data()
logging.info(f"Dados brutos carregados: {df_raw.shape[0]} registros, {df_raw.shape[1]} colunas")

# Mostrar colunas e amostra
print("\n🔍 Colunas disponíveis nos dados brutos:")
print(df_raw.columns.tolist())
print("\n📋 Amostra dos dados brutos:")
display(df_raw.head(3))
print("\n📊 Estatísticas descritivas:")
display(df_raw.describe())

# Pré-processamento
logging.info("Iniciando pré-processamento...")
preprocessor = DataPreprocessor()
df = preprocessor.preprocess(df_raw)
logging.info(f"Pré-processamento concluído: {df.shape[0]} registros, {df.shape[1]} colunas")

# Mostrar colunas e amostra após pré-processamento
print("\n🔍 Colunas disponíveis após pré-processamento:")
print(df.columns.tolist())
print("\n📋 Amostra dos dados processados:")
display(df.head(3))

# %% [markdown]
# ## 2. Análise Temporal Completa

# %%
if 'data' in df.columns:
    # Converter data para datetime
    df['data'] = pd.to_datetime(df['data'])
    
    # Criar figura com subplots
    fig, axs = plt.subplots(4, 1, figsize=(16, 20))
    
    # 2.1 Casos de arboviroses
    for disease in ['dengue', 'zika', 'chikungunya']:
        col = f'casos_{disease}'
        if col in df.columns:
            for city in df['municipio'].unique():
                city_data = df[df['municipio'] == city]
                axs[0].plot(city_data['data'], city_data[col], 'o-', label=f"{city} - {disease}")
    axs[0].set_title('Evolução de Casos de Arboviroses')
    axs[0].set_ylabel('Casos')
    axs[0].legend(ncol=2)
    axs[0].grid(True)
    
    # 2.2 Temperatura e umidade
    if 'temperatura' in df.columns:
        for city in df['municipio'].unique():
            city_data = df[df['municipio'] == city]
            axs[1].plot(city_data['data'], city_data['temperatura'], 's-', label=f"{city} - Temperatura")
    if 'umidade' in df.columns:
        for city in df['municipio'].unique():
            city_data = df[df['municipio'] == city]
            axs[1].plot(city_data['data'], city_data['umidade'], 'd-', label=f"{city} - Umidade")
    axs[1].set_title('Variáveis Climáticas: Temperatura e Umidade')
    axs[1].set_ylabel('Valor')
    axs[1].legend(ncol=2)
    axs[1].grid(True)
    
    # 2.3 Precipitação
    if 'precipitacao' in df.columns:
        for city in df['municipio'].unique():
            city_data = df[df['municipio'] == city]
            # Média móvel de 7 dias para suavizar
            axs[2].plot(city_data['data'], city_data['precipitacao'].rolling(7).mean(), 
                       '^-', label=f"{city} - Precipitação (mm7d)")
    axs[2].set_title('Precipitação (Média Móvel 7 dias)')
    axs[2].set_ylabel('mm')
    axs[2].legend()
    axs[2].grid(True)
    
    # 2.4 Relação entre precipitação e casos (lag 14 dias)
    if 'precipitacao' in df.columns and 'casos_dengue' in df.columns:
        for city in df['municipio'].unique():
            city_data = df[df['municipio'] == city]
            # Precipitação com lag de 14 dias
            axs[3].plot(city_data['data'], city_data['precipitacao'].shift(14).rolling(7).mean(), 
                       ':', label=f"{city} - Precipitação (lag 14d)")
            axs[3].plot(city_data['data'], city_data['casos_dengue'].rolling(7).mean(), 
                       '-', label=f"{city} - Casos Dengue (média 7d)")
        axs[3].set_title('Relação entre Precipitação (lag 14d) e Casos de Dengue')
        axs[3].set_ylabel('Valor Normalizado')
        axs[3].legend()
        axs[3].grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Coluna 'data' não encontrada. Análise temporal ignorada.")

# %% [markdown]
# ## 3. Análise de Correlação Avançada

# %%
# 3.1 Mapa de calor de correlações
corr_cols = ['temperatura', 'umidade', 'precipitacao', 
             'casos_dengue', 'casos_zika', 'casos_chikungunya']

# Filtrar colunas existentes
corr_cols = [col for col in corr_cols if col in df.columns]

if len(corr_cols) > 1:
    plt.figure(figsize=(12, 10))
    corr_matrix = df[corr_cols].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
    plt.title('Matriz de Correlação')
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Colunas insuficientes para matriz de correlação")

# 3.2 Correlação com lags
if 'precipitacao' in df.columns and 'casos_dengue' in df.columns:
    lags = np.arange(0, 31, 5)  # 0 a 30 dias
    correlations = []
    
    for lag in lags:
        corr = df['casos_dengue'].corr(df['precipitacao'].shift(lag))
        correlations.append(corr)
    
    plt.figure(figsize=(12, 6))
    plt.plot(lags, correlations, 'o-', markersize=8)
    plt.axhline(0, color='gray', linestyle='--')
    plt.title('Correlação entre Precipitação (lag) e Casos de Dengue')
    plt.xlabel('Dias de Atraso (lag)')
    plt.ylabel('Coeficiente de Correlação')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ## 4. Análise de Sazonalidade

# %%
if 'mes' in df.columns:
    # Agrupar por mês
    monthly_data = df.groupby('mes').agg({
        'casos_dengue': 'mean',
        'casos_zika': 'mean',
        'casos_chikungunya': 'mean',
        'temperatura': 'mean',
        'umidade': 'mean',
        'precipitacao': 'sum'
    }).reset_index()
    
    # Mapear nomes dos meses
    meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
             'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
    
    # 4.1 Casos por mês
    plt.figure(figsize=(14, 7))
    for disease in ['dengue', 'zika', 'chikungunya']:
        col = f'casos_{disease}'
        if col in monthly_data.columns:
            plt.plot(monthly_data['mes'], monthly_data[col], 'o-', label=disease.capitalize(), markersize=8)
    plt.title('Média Mensal de Casos de Arboviroses')
    plt.ylabel('Casos')
    plt.xlabel('Mês')
    plt.xticks(range(1, 13), meses)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # 4.2 Clima por mês
    plt.figure(figsize=(14, 10))
    
    plt.subplot(3, 1, 1)
    plt.plot(monthly_data['mes'], monthly_data['temperatura'], 's-', color='red', markersize=8)
    plt.title('Temperatura Média Mensal')
    plt.ylabel('°C')
    plt.xticks(range(1, 13), meses)
    plt.grid(True)
    
    plt.subplot(3, 1, 2)
    plt.plot(monthly_data['mes'], monthly_data['umidade'], 'd-', color='blue', markersize=8)
    plt.title('Umidade Média Mensal')
    plt.ylabel('%')
    plt.xticks(range(1, 13), meses)
    plt.grid(True)
    
    plt.subplot(3, 1, 3)
    plt.bar(monthly_data['mes'], monthly_data['precipitacao'], color='green')
    plt.title('Precipitação Acumulada Mensal')
    plt.ylabel('mm')
    plt.xticks(range(1, 13), meses)
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Coluna 'mes' não encontrada. Análise de sazonalidade ignorada.")

# %% [markdown]
# ## 5. Comparação entre Municípios

# %%
if 'municipio' in df.columns:
    # 5.1 Casos totais
    total_cases = df.groupby('municipio').agg({
        'casos_dengue': 'sum',
        'casos_zika': 'sum',
        'casos_chikungunya': 'sum'
    })
    
    if not total_cases.empty:
        plt.figure(figsize=(14, 8))
        total_cases.plot(kind='bar', stacked=True)
        plt.title('Casos Totais por Município')
        plt.ylabel('Casos')
        plt.xlabel('Município')
        plt.xticks(rotation=0)
        plt.grid(True, axis='y')
        plt.tight_layout()
        plt.show()
    
    # 5.2 Médias climáticas
    climate_means = df.groupby('municipio').agg({
        'temperatura': 'mean',
        'umidade': 'mean',
        'precipitacao': 'mean'
    })
    
    if not climate_means.empty:
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))
        
        axs[0].bar(climate_means.index, climate_means['temperatura'], color='red')
        axs[0].set_title('Temperatura Média')
        axs[0].set_ylabel('°C')
        
        axs[1].bar(climate_means.index, climate_means['umidade'], color='blue')
        axs[1].set_title('Umidade Média')
        axs[1].set_ylabel('%')
        
        axs[2].bar(climate_means.index, climate_means['precipitacao'], color='green')
        axs[2].set_title('Precipitação Média Diária')
        axs[2].set_ylabel('mm')
        
        for ax in axs:
            ax.grid(True, axis='y')
        
        plt.tight_layout()
        plt.show()
else:
    logging.warning("⚠️ Coluna 'municipio' não encontrada. Comparação entre municípios ignorada.")

# %% [markdown]
# ## 6. Análise de Distribuição e Outliers

# %%
# 6.1 Distribuição de casos
case_cols = [col for col in df.columns if 'casos_' in col]

if case_cols:
    plt.figure(figsize=(14, 8))
    for i, col in enumerate(case_cols, 1):
        plt.subplot(1, len(case_cols), i)
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribuição de {col.replace("casos_", "").capitalize()}')
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Nenhuma coluna de casos encontrada para análise de distribuição")

# 6.2 Boxplots climáticos
climate_cols = [col for col in ['temperatura', 'umidade', 'precipitacao'] if col in df.columns]

if climate_cols:
    plt.figure(figsize=(14, 6))
    sns.boxplot(data=df[climate_cols], orient='h')
    plt.title('Distribuição de Variáveis Climáticas')
    plt.grid(True, axis='x')
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Nenhuma coluna climática encontrada para análise de distribuição")

# %% [markdown]
# ## 7. Análise de Tendências

# %%
if 'data' in df.columns and 'casos_dengue' in df.columns:
    # Converter data para ordinal para regressão
    df['data_ordinal'] = df['data'].apply(lambda x: x.toordinal())
    
    # Calcular tendência para cada município
    for city in df['municipio'].unique():
        city_data = df[df['municipio'] == city]
        
        # Regressão linear para casos de dengue
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            city_data['data_ordinal'], 
            city_data['casos_dengue']
        )
        
        plt.figure(figsize=(12, 6))
        plt.scatter(city_data['data'], city_data['casos_dengue'], alpha=0.5, label='Casos Diários')
        
        # Linha de tendência
        trend_line = intercept + slope * city_data['data_ordinal']
        plt.plot(city_data['data'], trend_line, 'r-', linewidth=2, 
                label=f'Tendência (R²={r_value**2:.2f})')
        
        plt.title(f'Tendência de Casos de Dengue em {city}')
        plt.ylabel('Casos')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
else:
    logging.warning("⚠️ Dados insuficientes para análise de tendências")

# %% [markdown]
# ## 8. Visualização Espacial (Mapa de Calor Temporal)

# %%
if 'data' in df.columns and 'municipio' in df.columns and 'casos_dengue' in df.columns:
    # Preparar dados para heatmap
    heatmap_data = df.pivot_table(
        index='data', 
        columns='municipio', 
        values='casos_dengue', 
        aggfunc='sum'
    ).fillna(0)
    
    # Média móvel semanal
    heatmap_data = heatmap_data.rolling(7).mean()
    
    # Plotar heatmap
    plt.figure(figsize=(14, 10))
    sns.heatmap(
        heatmap_data.T, 
        cmap='YlOrRd',
        cbar_kws={'label': 'Casos de Dengue (média 7 dias)'}
    )
    plt.title('Evolução Temporal de Casos de Dengue por Município')
    plt.ylabel('Município')
    plt.xlabel('Data')
    plt.tight_layout()
    plt.show()
else:
    logging.warning("⚠️ Dados insuficientes para mapa de calor")

# %% [markdown]
# ## Conclusões e Próximos Passos
# 
# Com base nas análises realizadas, podemos:
# 
# 1. Identificar padrões sazonais nas doenças
# 2. Correlacionar variáveis climáticas com incidência de casos
# 3. Detectar municípios com maior carga de doenças
# 4. Identificar tendências temporais
# 
# **Próximos passos:**
# - Desenvolver modelos preditivos
# - Implementar sistema de alerta precoce
# - Analisar impacto de intervenções

# %%
# Salvar dados processados para uso futuro
df.to_csv('data/processed/dados_arboviroses_processados.csv', index=False)
logging.info("Dados processados salvos em 'data/processed/dados_arboviroses_processados.csv'")

ModuleNotFoundError: No module named 'src'